# Interactive Corporate Credit Report Generator (Simulated)

This notebook allows you to input key information for a company and then generates:
1.  A structured **LLM Prompt** that could be sent to a powerful language model.
2.  A **Simulated Credit Report** in Markdown format, dynamically generated based on your inputs,
    using a local Python-based simulation logic (does not call an actual LLM).

**Instructions:**
1. Run the setup cells below (Cells 2, 3, 4).
2. Scroll down to the generated UI form.
3. Fill in or modify the input fields with company-specific information.
4. Click the "Generate Full Prompt & Simulated Report" button.
5. Review the generated LLM prompt and the simulated report displayed at the bottom.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML
import json # Not strictly needed for this version of UI config, but good to have
import datetime # For DynamicReportSimulator if it uses it (it does not in provided example)
import re # For DynamicReportSimulator

# --- Content of prompt_template_core.txt ---
PROMPT_TEMPLATE_CORE = """
You are an expert senior credit analyst AI, tasked with generating a comprehensive, balanced, and insightful corporate credit report. Your analysis should be objective and data-driven, drawing upon the information provided below.

**Objective:** Produce a corporate credit report for the specified company, adhering to the structure outlined in the "REPORT_STRUCTURE_GUIDE".

**Key Instructions:**
1.  **Company Focus:** The report is for: {company_name} ({company_ticker}), operating in the {company_sector} sector.
2.  **Information Provided by User:** You will receive structured inputs including:
    * Key Financial Data
    * Calculated Credit Metrics
    * Qualitative Assessments (Management, Competitive Landscape, Industry Outlook, ESG Factors)
    * Recent News Snippets / Press Releases
    * Analyst's Key Assumptions
3.  **Analysis Approach:**
    * Synthesize all provided quantitative and qualitative information.
    * Identify key credit strengths and weaknesses.
    * Discuss financial performance and creditworthiness based on the data.
    * Incorporate recent developments and their potential impact.
    * Clearly state the rating rationale and outlook based *only* on the information provided.
    * If specific data for a standard report section is NOT provided, explicitly state "Information not provided for this section." or "Analysis for this section is limited due to lack of specific input." Do NOT invent data.
4.  **Tone:** Professional, analytical, objective, and cautious. Use clear and concise language.
5.  **Output Format:** Generate the report in Markdown format, strictly following the section headers and structure provided in the "REPORT_STRUCTURE_GUIDE".
6.  **Disclaimer:** Conclude the report with the mandatory disclaimer: "This report is a simulated analysis generated based on user-provided inputs and should not be used for actual investment or credit decisions. Verify all information independently."

**USER-PROVIDED INFORMATION WILL BE INSERTED BELOW THIS LINE WHEN THE FULL PROMPT IS CONSTRUCTED.**
---INPUT_DATA_MARKER---
**REPORT_STRUCTURE_GUIDE:**
{report_structure_guide}
---END_REPORT_STRUCTURE_GUIDE---
**FINAL INSTRUCTION: Now, generate the comprehensive corporate credit report based on all the above instructions and the provided user inputs, adhering strictly to the REPORT_STRUCTURE_GUIDE.**
"""

# --- Content of report_structure_guide.md ---
REPORT_STRUCTURE_GUIDE = """
# Corporate Credit Report: {company_name} ({company_ticker})

## 1. Executive Summary
    * Overall Assessment: {overall_assessment_placeholder}
    * Simulated Credit Rating: {simulated_credit_rating_placeholder}
    * Rating Outlook: {rating_outlook_placeholder}
    * Key Positive Factors: 
        * {key_positive_factors_placeholder}
    * Key Credit Concerns:
        * {key_credit_concerns_placeholder}

## 2. Company Overview
    * Company Name: {company_name}
    * Ticker Symbol: {company_ticker}
    * Primary Sector: {company_sector}
    * Brief Business Description (if provided by user): {qualitative_business_description}

## 3. Key Analyst Assumptions
    * {key_assumptions}

## 4. Financial Performance Analysis
    * Summary of Provided Financials:
        * {financial_data_summary}
    * Key Credit Metrics Analysis:
        * {credit_metrics_summary}
    * Trend Analysis (based on provided data interpretation):
        * {trend_analysis_placeholder}

## 5. Qualitative Factors Assessment
    * Management & Strategy:
        * {qualitative_management_strategy}
    * Competitive Landscape & Market Position:
        * {qualitative_competitive_landscape}
    * Industry Outlook:
        * {qualitative_industry_outlook}
    * Environmental, Social, and Governance (ESG) Considerations (if provided):
        * {qualitative_esg_factors}

## 6. Recent Developments & News
    * Summary of Recent Press Releases/News:
        * {recent_news_summary}
    * Potential Impact Analysis (based on analyst input or LLM inference if obvious from news):
        * {news_impact_analysis_placeholder}

## 7. Credit Strengths
    * {credit_strengths_placeholder}

## 8. Credit Risks & Concerns
    * {credit_risks_concerns_placeholder}

## 9. Rating Rationale
    * {rating_rationale_placeholder}

## 10. Outlook Rationale
    * {outlook_rationale_placeholder}

## 11. Disclaimer
This report is a simulated analysis generated based on user-provided inputs and should not be used for actual investment or credit decisions. Verify all information independently.
"""

# --- Content of input_ui_config.json (as a Python dict) ---
INPUT_UI_CONFIG = { # Copied directly from user's prompt
  "sections": [
    { "title": "Company Information", "fields": [
        {"name": "company_name", "label": "Company Name:", "type": "text", "default": "ExampleCorp"},
        {"name": "company_ticker", "label": "Ticker Symbol:", "type": "text", "default": "EXMPL"},
        {"name": "company_sector", "label": "Primary Sector:", "type": "text", "default": "Technology"}
    ]},
    { "title": "Key Analyst Assumptions", "fields": [
        {"name": "key_assumptions", "label": "Analyst's Key Assumptions:", "type": "textarea", "default": "1. Moderate revenue growth in line with industry.\n2. Stable margins.\n3. No major M&A activity in the next 12 months."}
    ]},
    { "title": "Financial Data (Illustrative)", "fields": [
        {"name": "financial_revenue_y1", "label": "Recent Full Year Revenue ($M):", "type": "float", "default": 1000.0},
        {"name": "financial_ebitda_y1", "label": "Recent Full Year EBITDA ($M):", "type": "float", "default": 250.0},
        {"name": "financial_total_debt_y1", "label": "Total Debt ($M):", "type": "float", "default": 500.0},
        {"name": "financial_total_equity_y1", "label": "Total Equity ($M):", "type": "float", "default": 300.0},
        {"name": "financial_fcf_y1", "label": "Free Cash Flow ($M):", "type": "float", "default": 50.0}
    ]},
    { "title": "Credit Metrics (Illustrative)", "fields": [
        {"name": "metric_debt_ebitda", "label": "Debt / EBITDA (x):", "type": "float", "default": 2.0},
        {"name": "metric_ebitda_interest", "label": "EBITDA / Interest Expense (x):", "type": "float", "default": 5.0}
    ]},
    { "title": "Qualitative Factors", "fields": [
        {"name": "qualitative_business_description", "label": "Brief Business Description:", "type": "textarea", "default": "Leading provider of innovative widgets and associated services."},
        {"name": "qualitative_management_strategy", "label": "Management Assessment & Strategy:", "type": "textarea", "default": "Experienced management team with a clear strategic focus on core markets and product innovation. Execution track record is strong."},
        {"name": "qualitative_competitive_landscape", "label": "Competitive Landscape & Market Position:", "type": "textarea", "default": "Operates in a competitive market but holds a significant market share in its primary niche. Strong brand recognition."},
        {"name": "qualitative_industry_outlook", "label": "Industry Outlook:", "type": "textarea", "default": "Industry expected to see modest growth, with potential headwinds from supply chain disruptions."},
        {"name": "qualitative_esg_factors", "label": "ESG Considerations:", "type": "textarea", "default": "Company has a published ESG report outlining initiatives in carbon reduction and diversity. No major controversies noted."}
    ]},
    { "title": "Recent News / Press Releases", "fields": [
        {"name": "recent_news_summary", "label": "Paste recent news snippets/summaries here:", "type": "textarea", "default": "ExampleCorp announces partnership with TechGiant for new product line (Source: PR Newswire, 2025-05-01).\nQ1 earnings slightly above expectations despite macro headwinds (Source: Earnings Call Transcript, 2025-04-15)."}
    ]}
  ]
}
print("Setup cell executed: Templates and UI Config loaded.")

In [ ]:
class DynamicReportSimulator:
    def __init__(self, report_structure_template_md):
        self.report_structure_template_md = report_structure_template_md

    def _format_financials(self, inputs_dict):
        summary = []
        for key, value in inputs_dict.items():
            if key.startswith('financial_') and value is not None:
                label = key.replace('financial_', '').replace('_', ' ').title()
                unit = "($M)" if any(term in label for term in ["Revenue", "Ebitda", "Debt", "Equity", "Fcf"]) else ""
                summary.append(f"- {label}: {value}{unit if unit else ''}")
        return "\n        ".join(summary) if summary else "No specific financial figures provided."

    def _format_metrics(self, inputs_dict):
        summary = []
        commentary = []
        for key, value in inputs_dict.items():
            if key.startswith('metric_') and value is not None:
                label = key.replace('metric_', '').replace('_', ' ').upper()
                summary.append(f"- {label}: {value}x")
                try: # Add try-except for float conversion
                    val_float = float(value)
                    if 'DEBT / EBITDA' in label:
                        if val_float <= 2.0: commentary.append("Leverage (Debt/EBITDA) appears low to moderate.")
                        elif val_float <= 4.0: commentary.append("Leverage (Debt/EBITDA) appears moderate.")
                        else: commentary.append("Leverage (Debt/EBITDA) appears high.")
                    if 'EBITDA / INTEREST' in label: # Corrected label check
                        if val_float >= 5.0: commentary.append("Interest coverage appears strong.")
                        elif val_float >= 2.0: commentary.append("Interest coverage appears adequate.")
                        else: commentary.append("Interest coverage appears weak.")
                except ValueError:
                    commentary.append(f"Could not parse metric value for {label} for commentary.")


        metrics_text = "\n        ".join(summary) if summary else "No specific credit metrics provided."
        if commentary:
            metrics_text += "\n    * **Brief Commentary:**\n        * " + "\n        * ".join(commentary)
        return metrics_text

    def _infer_rating_outlook(self, inputs_dict):
        rating = "BBB"
        outlook = "Stable"
        score = 0
        try:
            if float(inputs_dict.get('financial_fcf_y1', 0.0)) > 0: score += 1
            else: score -=1
            if float(inputs_dict.get('financial_revenue_y1', 0.0)) > 500 : score +=1
            
            debt_ebitda = inputs_dict.get('metric_debt_ebitda')
            if debt_ebitda is not None:
                val_de = float(debt_ebitda)
                if val_de < 1.5: score += 2
                elif val_de < 3.0: score += 1
                elif val_de > 4.5: score -= 2
                else: score -=1
            
            ebitda_interest = inputs_dict.get('metric_ebitda_interest')
            if ebitda_interest is not None:
                val_ei = float(ebitda_interest)
                if val_ei > 8.0: score += 2
                elif val_ei > 4.0: score += 1
                elif val_ei < 2.0: score -=2
                else: score -=1

            qual_texts = [
                str(inputs_dict.get('qualitative_management_strategy','')),
                str(inputs_dict.get('qualitative_competitive_landscape','')),
                str(inputs_dict.get('qualitative_industry_outlook',''))
            ]
            negative_keywords = ["poor", "declining", "weak", "intense competition", "headwinds", "challenging"]
            for text in qual_texts:
                for keyword in negative_keywords:
                    if keyword in text.lower(): score -=1; break
            if "strong growth" in str(inputs_dict.get('qualitative_industry_outlook','')).lower() : score +=1
            if "strong execution" in str(inputs_dict.get('qualitative_management_strategy','')).lower() : score +=1
        except (ValueError, TypeError) as e:
            print(f"Warning: Error during rating inference due to input type: {e}")
            # Keep default rating/outlook if inputs are problematic
        
        if score >= 4: rating = "A-"; outlook = "Positive"
        elif score >= 2: rating = "BBB+"; outlook = "Stable"
        elif score >= 0: rating = "BBB"; outlook = "Stable"
        elif score >= -2: rating = "BBB-"; outlook = "Negative"
        elif score >= -4: rating = "BB+"; outlook = "Negative"
        else: rating = "BB"; outlook = "Negative"
        return rating, outlook

    def generate_simulated_report(self, inputs_dict):
        sim_rating, sim_outlook = self._infer_rating_outlook(inputs_dict)
        
        positive_factors = []
        credit_concerns = []
        try:
            if float(inputs_dict.get('financial_fcf_y1', 0.0)) > 0:
                positive_factors.append(f"Positive Free Cash Flow (${inputs_dict.get('financial_fcf_y1', 'N/A')}M).")
            else:
                credit_concerns.append(f"Negative or low Free Cash Flow (${inputs_dict.get('financial_fcf_y1', 'N/A')}M).")

            debt_ebitda = inputs_dict.get('metric_debt_ebitda')
            if debt_ebitda is not None:
                val_de = float(debt_ebitda)
                if val_de < 2.0: positive_factors.append(f"Low leverage (Debt/EBITDA: {val_de}x).")
                elif val_de > 4.0: credit_concerns.append(f"High leverage (Debt/EBITDA: {val_de}x).")

            if "strong execution" in str(inputs_dict.get('qualitative_management_strategy','')).lower():
                positive_factors.append("Indication of strong management execution.")
            if "intense competition" in str(inputs_dict.get('qualitative_competitive_landscape','')).lower():
                credit_concerns.append("Intense competitive landscape noted.")
        except (ValueError, TypeError) as e:
             positive_factors.append(f"(Error processing factors: {e})")
             credit_concerns.append(f"(Error processing factors: {e})")

        if not positive_factors: positive_factors.append("No specific positive factors strongly highlighted from inputs.")
        if not credit_concerns: credit_concerns.append("No specific credit concerns strongly highlighted from inputs.")

        report_replacements = {
            'company_name': str(inputs_dict.get('company_name', 'N/A')),
            'company_ticker': str(inputs_dict.get('company_ticker', 'N/A')),
            'company_sector': str(inputs_dict.get('company_sector', 'N/A')),
            'qualitative_business_description': str(inputs_dict.get('qualitative_business_description', 'Information not provided.')),
            'key_assumptions': str(inputs_dict.get('key_assumptions', 'Information not provided.')).replace('\n', '\n    * '),
            'financial_data_summary': self._format_financials(inputs_dict),
            'credit_metrics_summary': self._format_metrics(inputs_dict),
            'qualitative_management_strategy': str(inputs_dict.get('qualitative_management_strategy', 'Information not provided.')),
            'qualitative_competitive_landscape': str(inputs_dict.get('qualitative_competitive_landscape', 'Information not provided.')),
            'qualitative_industry_outlook': str(inputs_dict.get('qualitative_industry_outlook', 'Information not provided.')),
            'qualitative_esg_factors': str(inputs_dict.get('qualitative_esg_factors', 'Information not provided.')),
            'recent_news_summary': str(inputs_dict.get('recent_news_summary', 'Information not provided.')).replace('\n', '\n        * '),
            'simulated_credit_rating_placeholder': sim_rating,
            'rating_outlook_placeholder': sim_outlook,
            'key_positive_factors_placeholder': "\n        * ".join(positive_factors),
            'key_credit_concerns_placeholder': "\n        * ".join(credit_concerns),
            'overall_assessment_placeholder': f"Overall assessment suggests a credit profile consistent with a {sim_rating} rating and a {sim_outlook} outlook. Key strengths appear to be [{positive_factors[0] if positive_factors else 'N/A'}], while key concerns include [{credit_concerns[0] if credit_concerns else 'N/A'}]. Financial metrics show leverage at {inputs_dict.get('metric_debt_ebitda', 'N/A')}x and interest coverage at {inputs_dict.get('metric_ebitda_interest', 'N/A')}x.",
            'rating_rationale_placeholder': f"The simulated rating of {sim_rating} is based on factors such as financial health (e.g., FCF: ${inputs_dict.get('financial_fcf_y1', 'N/A')}M, Debt/EBITDA: {inputs_dict.get('metric_debt_ebitda', 'N/A')}x), market position ('{str(inputs_dict.get('qualitative_competitive_landscape', ''))[:70]}...'), and management ('{str(inputs_dict.get('qualitative_management_strategy', ''))[:70]}...').",
            'outlook_rationale_placeholder': f"The {sim_outlook} outlook reflects assumptions ('{str(inputs_dict.get('key_assumptions', ''))[:70]}...'), industry trends ('{str(inputs_dict.get('qualitative_industry_outlook', ''))[:70]}...'), and recent news ('{str(inputs_dict.get('recent_news_summary', ''))[:70]}...').",
            'trend_analysis_placeholder': "[Detailed trend analysis requires time-series data. Based on single-period input, focus is on current state.]",
            'news_impact_analysis_placeholder': f"[News ('{str(inputs_dict.get('recent_news_summary', ''))[:70]}...') could impact credit by [e.g., altering FCF, leverage, or market perception]. Further analysis needed.]",
            'credit_strengths_placeholder': "* " + "\n        * ".join(positive_factors),
            'credit_risks_concerns_placeholder': "* " + "\n        * ".join(credit_concerns)
        }
        
        report_md = self.report_structure_template_md
        for key, value in report_replacements.items():
            report_md = report_md.replace(f"{{{key}}}", str(value))
        return report_md

report_simulator = DynamicReportSimulator(REPORT_STRUCTURE_GUIDE)
print("DynamicReportSimulator class defined and instance created.")

In [ ]:
input_widgets = {}
sections_vbox_list = []

for section_config in INPUT_UI_CONFIG['sections']: # Renamed to avoid conflict
    section_title_html = widgets.HTML(f"<h3>{section_config['title']}</h3>")
    fields_vbox_list = [section_title_html]
    for field_config in section_config['fields']:
        label_widget = widgets.Label(field_config['label']) # Renamed to avoid conflict
        
        widget_kwargs = {'value': field_config['default']}
        if field_config['type'] == 'textarea':
            widget_kwargs['layout'] = widgets.Layout(width='90%', height='100px')
            widget = widgets.Textarea(**widget_kwargs)
        elif field_config['type'] == 'float':
            widget_kwargs['layout'] = widgets.Layout(width='auto')
            widget = widgets.FloatText(**widget_kwargs)
        else: # Default to text
            widget_kwargs['layout'] = widgets.Layout(width='90%')
            widget = widgets.Text(**widget_kwargs)
        
        input_widgets[field_config['name']] = widget
        # Use HBox for label and widget to be side-by-side, then VBox for each field row for alignment
        fields_vbox_list.append(widgets.VBox([label_widget, widget]))
    sections_vbox_list.append(widgets.VBox(fields_vbox_list, layout=widgets.Layout(margin='0 0 15px 0')))

all_inputs_vbox = widgets.VBox(sections_vbox_list)
print("UI widgets generated.")

In [ ]:
generate_button = widgets.Button(
    description="Generate Full Prompt & Simulated Report",
    button_style='success',
    layout=widgets.Layout(width='auto', margin='20px 0 20px 0')
)
output_area = widgets.Output()

def on_generate_button_clicked(b):
    with output_area:
        output_area.clear_output(wait=True)
        display(HTML("<h4>Processing...</h4>"))
        
        user_inputs = {name: w.value for name, w in input_widgets.items()}
        
        input_data_prompt_section = "**USER-PROVIDED INFORMATION:**\n\n"
        # Simplified loop for prompt data section
        for section_cfg in INPUT_UI_CONFIG['sections']:
            input_data_prompt_section += f"**{section_cfg['title']}**\n"
            for field_cfg in section_cfg['fields']:
                field_name = field_cfg['name']
                field_label = field_cfg['label']
                # Get value, ensuring it's a string for prompt
                field_value_str = str(user_inputs.get(field_name, 'N/A'))
                if field_cfg['type'] == 'textarea': # Add some formatting for textareas
                     field_value_str = "\n".join([f"  {line}" for line in field_value_str.split('\n')])
                input_data_prompt_section += f"- {field_label} {field_value_str}\n"
            input_data_prompt_section += "\n"

        full_llm_prompt = PROMPT_TEMPLATE_CORE.replace("{company_name}", str(user_inputs.get('company_name', 'N/A')))
        full_llm_prompt = full_llm_prompt.replace("{company_ticker}", str(user_inputs.get('company_ticker', 'N/A')))
        full_llm_prompt = full_llm_prompt.replace("{company_sector}", str(user_inputs.get('company_sector', 'N/A')))
        full_llm_prompt = full_llm_prompt.replace("---INPUT_DATA_MARKER---", input_data_prompt_section + "\n---\n")
        full_llm_prompt = full_llm_prompt.replace("{report_structure_guide}", REPORT_STRUCTURE_GUIDE)

        display(HTML("<h2>Generated LLM Prompt:</h2>"))
        display(Markdown(f"```text\n{full_llm_prompt}\n```"))
        
        display(HTML("<h2>Simulated Credit Report:</h2>"))
        simulated_report_md = report_simulator.generate_simulated_report(user_inputs)
        display(Markdown(simulated_report_md))

generate_button.on_click(on_generate_button_clicked)
print("Button logic defined.")

In [ ]:
display(HTML("<h1>Interactive Credit Report Input Form</h1>"))
display(all_inputs_vbox)
display(generate_button)
display(output_area)
print("UI displayed. Please fill the form and click generate.")